In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [12]:
def one_hot(input_df):
    df = input_df.copy()
    cols = list(df)
    for col in cols:
        if df[col].dtype == "object":
            df = pd.get_dummies(df, columns=[col], prefix=col)
    return df

In [39]:
train = pd.read_csv("hp_train.csv")
test = pd.read_csv("hp_train.csv")

In [74]:
dataset = pd.concat(objs=[train.copy(), test.copy()], axis=0).reset_index(drop=True)
dataset = dataset.fillna(np.nan)
dataset = dataset.fillna(0)
dataset = one_hot(dataset)

In [75]:
train_len = len(train)

x_train = dataset[:train_len]

f_test = dataset[train_len:].drop('SalePrice', axis=1)

y_train = x_train["SalePrice"]
x_train = x_train.drop("SalePrice", axis=1)

In [9]:
cols = list(x_train)
for col in cols:
    if x_train[col].dtype == "float64":
        print(col)

LotFrontage
MasVnrArea
GarageYrBlt


In [16]:
rfc = RandomForestClassifier()
parms = {
    "n_estimators" : [4, 6, 8],
    "max_features" : ["log2", "sqrt", "auto"],
    "criterion" : ["entropy", "gini"],
    "max_depth" : [2, 3, 5],
    "min_samples_split" : [2, 3, 5],
    "min_samples_leaf" : [1, 5, 8],
    "oob_score" : [True],
    "random_state" : [7]
}
acc_scorer = make_scorer(accuracy_score)
gs = GridSearchCV(rfc, parms, scoring=acc_scorer)
gs = gs.fit(x_train, y_train)
rfc = gs.best_estimator_
print(rfc.score(x_train, y_train))

0.0315068493151


In [77]:
pred = rfc.predict(f_test)

output_hp = f_test['Id'].to_frame('Id')
output_hp = output_hp.assign(SalePrice=pred)
output_hp.to_csv('output_hp.csv', index=False)